# Kitas in Hannover #

Welche Kitas gibt es in Hannover überhaupt und wo befinden sie sich?

http://www.hannover.de/Leben-in-der-Region-Hannover/Soziales/Kinder-Jugendliche/Kinderbetreuung/Kindertagesst%C3%A4tten-in-der-Stadt-Hannover/Kitas-in-Hannover-nach-Tr%C3%A4gern/Kitas-der-Landeshauptstadt-Hannover

### Webcrawling in Python ###

In [1]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

geolocator = Nominatim()

def parse_page(thePage):
    line_divs = thePage.find_all("div", class_="line_view")
    results = []
    for div in line_divs:
        name = div.a['title']
        split = name.split(" - ")
        address = div.a.div.p.contents[0].strip()

        location = geolocator.geocode(address)
        if(location is not None):
            #print("'" + name + "', " + str(location.latitude) + ", " + str(location.longitude))
            results.append((split[0], split[1], location))
        else:
            print("*** NO RESULT FOR: '" + address + "' ***")
        
    return(results)

In [4]:
baseurl = "http://www.hannover.de"
url = baseurl + "/Leben-in-der-Region-Hannover/Soziales/Kinder-Jugendliche/Kinderbetreuung/Kindertagesst%C3%A4tten-in-der-Stadt-Hannover/Kitas-in-Hannover-nach-Tr%C3%A4gern/Kitas-der-Landeshauptstadt-Hannover"

kita_infos = []
nextPage = True
while(nextPage):
    dom = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(dom, 'html.parser')
    results = parse_page(soup)
    kita_infos.extend(results)
    
    next = soup.find_all("span", class_="next")
    link = next[0].a
    
    if link is not None:
        text = link.get("href", None)
        url = baseurl + urllib.parse.quote(text)
    else:
        nextPage = False

print('Found ' + str(len(kita_infos)) + ' Kitas.')

*** NO RESULT FOR: 'Rötngenstraße 10, 30163 Hannover' ***
*** NO RESULT FOR: 'Gronostraße 9 C-E, 30459 Hannover' ***
*** NO RESULT FOR: 'Isernhagener Straße 82, 30165 Hannover' ***
Found 43 Kitas.


### Kartenvisualisierung in Python ###

In [5]:
from IPython.display import HTML
import folium

def embed_map(map, path="map.html"):
    map.save(outfile=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

ha_map = folium.Map(location=[52.3770, 9.7404], zoom_start=12)
for kita in kita_infos:
    folium.Marker([kita[2].latitude, kita[2].longitude], icon=folium.Icon(icon='info-sign'), popup=kita[1]).add_to(ha_map)
embed_map(ha_map, "ha_map.html")